In [1]:
#Required packages
import numpy as np
import torch
import torch.nn as nn

In [2]:
class NNet(nn.Module):
    #Initialize the model
    def __init__(self, in_channels=32, out_channels=6):
        super(NNet, self).__init__()
        #Number of hidden units
        self.hidden = 32
        #Model input layers
        self.net = nn.Sequential(
            nn.Conv1d(32, 32, 5, padding=2),
            nn.Conv1d(self.hidden, self.hidden, 16, stride=16),
            nn.LeakyReLU(0.1),
            nn.Conv1d(self.hidden, self.hidden, 7, padding=3),
        )
        #Add six convolutional blocks
        for i in range(6):
            self.net.add_module('conv{}'.format(i), \
                                self.__block(self.hidden, self.hidden))
        #Model output layer
        self.net.add_module('final', nn.Sequential(
            nn.Conv1d(self.hidden, out_channels, 1),
            nn.Sigmoid()
        ))
    #Method for convolutional block
    def __block(self, inchannels, outchannels):
        return nn.Sequential(
            nn.MaxPool1d(2, 2),
            nn.Dropout(p=0.1, inplace=True),
            nn.Conv1d(inchannels, outchannels, 5, padding=2),
            nn.LeakyReLU(0.1),
            nn.Conv1d(outchannels, outchannels, 5, padding=2),
            nn.LeakyReLU(0.1),
        )
    #Model forward pass
    def forward(self, x):
        return self.net(x)

In [4]:
#Create model architecture
trained_model = NNet()
#Load model weights from saved model file
trained_model.load_state_dict(torch.load('model2.pt'))
#Set model in evaluation mode
trained_model.eval()
#Convert model to double precision
trained_model = trained_model.to(dtype=torch.double)
#print model summary
print(trained_model)

NNet(
  (net): Sequential(
    (0): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): Conv1d(32, 32, kernel_size=(16,), stride=(16,))
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=(3,))
    (conv0): Sequential(
      (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): Dropout(p=0.1, inplace=True)
      (2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (3): LeakyReLU(negative_slope=0.1)
      (4): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (5): LeakyReLU(negative_slope=0.1)
    )
    (conv1): Sequential(
      (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): Dropout(p=0.1, inplace=True)
      (2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (3): LeakyReLU(negative_slope=0.1)
      (4): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (5): LeakyReLU(neg

In [38]:
#Load numpy array and perform model prediction
example_array = np.genfromtxt("example_array.csv", delimiter=",")
print(trained_model(torch.from_numpy(example_array)))

tensor([[0.0562],
        [0.0005],
        [0.0002],
        [0.0005],
        [0.0006],
        [0.0004]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
